In [14]:
%matplotlib inline
# %matplotlib notebook
import rasterio
from rasterio.coords import BoundingBox
from shapely.geometry import box
import geopandas as gpd
from rasterio.mask import mask
import os
import pandas as pd
import numpy as np

In [17]:
url_imagen = '/home/diego/Proyecto Vivero/data/01_VIVERO/01_imagenes/imagen_mesones_rgb.tif'
shp = '/home/diego/Proyecto Vivero/data/01_VIVERO/02_puntos_plantas/plantas_mesones_2.shp'
url_salida = "/home/diego/Proyecto Vivero/train2"

In [8]:
shape = gpd.read_file(shp)
raster = rasterio.open(url_imagen)

In [10]:
offsetX = 0
offsetY = 0
width = 100
height = 100
step = 10
cuentas = pd.DataFrame(columns=['imagen', 'cant_arboles'])

In [11]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [36]:
contador = 0
while (1):
        # finalizar si se a recorrido toda la imagen
    if(offsetY >= raster.height and offsetX >= raster.width):
        break
     #cuando llegue al borde en el eje x, debo bajar(sumar en eje y) y debo volver a empezar setear offsetX en 0 y widthX en 100
    elif(offsetX >= raster.width):
        offsetY = offsetY + step
        offsetX = 0
   
    #entregamos los pixeles para obtener las coordenadas
    t = raster.xy(offsetX, offsetY)
    l = raster.xy(offsetX + width , offsetY)
    r = raster.xy(offsetX, offsetY+ height)
    b = raster.xy(offsetX + width , offsetY + height)  
    bbox = box(l[0], b[1], r[0], t[1])
        
    try:
        #transformacion de la imagen
        out_img, out_transform = mask(raster, shapes=[bbox], crop=True)
        valores = np.unique(out_img)
        if np.array_equal(valores,[65535]):
            offsetX = offsetX + step
            contador = contador + 1
            continue
        out_meta = raster.meta.copy()
        out_meta.update(
        {"driver": "GTiff",
              "height": 100,
              "width": 100,
              "transform": out_transform,
              "crs": raster.crs})
        with rasterio.open(url_salida+"/salida"+str(contador)+".tif", "w", **out_meta) as dest:
            dest.write(out_img)

    #interseccion ptos
        cantidad=shape["geometry"].intersects(bbox).value_counts()
        if (True in cantidad.index):
            numero = cantidad[True]
        else:
            numero = 0
        cuentas.loc[contador] = ["salida"+str(contador)+".tif",numero]
        if(contador%1000 == 0):
            print("salida"+str(contador)+".tif")

        
    except ValueError:
        print(url_salida+"/salida"+str(contador)+".tif")
#         print(coords)
    contador = contador + 1
    offsetX = offsetX + step 
   

salida0.tif


KeyboardInterrupt: 

In [38]:
cuentas.to_csv(url_salida+"/cuentas.csv")

,imagen,cant_arboles
222,salida222.tif,0
223,salida223.tif,0
224,salida224.tif,0
32,salida32.tif,0
33,salida33.tif,0
